In [1]:
import numpy as np
import pandas as pd 
import os
import itertools

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
import pickle

%matplotlib inline

from sklearn.preprocessing import LabelBinarizer, LabelEncoder
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from tensorflow import keras

layers = keras.layers
models = keras.models

C:\Users\sulek\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
data = pd.read_csv("bbc-text.csv")
data.head()

,category,text
0,tech,tv future in the hands of viewers with home th...
1,business,worldcom boss left books alone former worldc...
2,sport,tigers wary of farrell gamble leicester say ...
3,sport,yeading face newcastle in fa cup premiership s...
4,entertainment,ocean s twelve raids box office ocean s twelve...


In [16]:
X = data['text']
y = data['category']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [17]:
max_words = 1000
tokenize = keras.preprocessing.text.Tokenizer(num_words=max_words, char_level=False)
tokenize.fit_on_texts(X_train) # fit tokenizer to our training text data
X_train = tokenize.texts_to_matrix(X_train)
X_test = tokenize.texts_to_matrix(X_test)
num_classes = len(set(y_train))

In [18]:
model = models.Sequential()
model.add(layers.Dense(512, input_shape=(max_words,)))
model.add(layers.Activation('relu'))
model.add(layers.Dense(num_classes))
model.add(layers.Activation('softmax'))

model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [19]:
encoder = LabelEncoder()
encoder.fit(y_train)
y_train = encoder.transform(y_train)
y_test = encoder.transform(y_test)


early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', 
                                     min_delta=0, patience=0, 
                                     verbose=0, mode='auto', 
                                     baseline=None)
history = model.fit(X_train, y_train,
                    batch_size = 32,
                    epochs = 2,
                    verbose=1,
                    callbacks = [early_stop],
                    validation_data = (X_test, y_test))

Train on 1780 samples, validate on 445 samples
Epoch 1/2
1780/1780 [==============================] - 2s 955us/step - loss: 0.3983 - acc: 0.8899 - val_loss: 0.1834 - val_acc: 0.9393
Epoch 2/2
1780/1780 [==============================] - 1s 827us/step - loss: 0.0326 - acc: 0.9983 - val_loss: 0.1564 - val_acc: 0.9506


In [20]:
data.head()

,category,text
0,tech,tv future in the hands of viewers with home th...
1,business,worldcom boss left books alone former worldc...
2,sport,tigers wary of farrell gamble leicester say ...
3,sport,yeading face newcastle in fa cup premiership s...
4,entertainment,ocean s twelve raids box office ocean s twelve...


In [21]:
model.predict(tokenize.texts_to_matrix(["India defeat new zealand by 1 run"])).argmax()

0

In [22]:
encoder.inverse_transform([0])

C:\Users\sulek\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


array(['business'], dtype=object)

LabelEncoder()

In [23]:
with open('tokenizer.pkl', 'wb') as handle:
    pickle.dump(tokenize, handle)
model.save('model.h5')
with open('encoder.pkl', 'wb') as handle:
    pickle.dump(encoder, handle)